# Tutorial for Real-Time fMRI Cloud-Based Framework

## Installation and Set Up

1. To run this tutorial, you must first install our **Real-Time fMRI Cloud Framework**, henceforth rtcloud framework. Installation instructions are found on our [github repo](https://github.com/brainiak/rt-cloud/blob/master/Readme.md#installation). Specifically, you should go through steps 1-2.

    * This will require cloning the rtcloud framework repo on your local computer (or wherever you are running this tutorial). We recommend that you clone the repo at the same level where you cloned or will clone the [github repo accompanying our Aperture paper](https://github.com/brainiak/brainiak-aperture/tree/master).


2. If you haven't done so already, you should clone the brainiak-aperture repo to your local computer or server (so don't forget to <code>cd ..</code> if necessary!). Your directory structure should look something like this:

```
.
├── brainiak-aperture
│   ├── LICENSE
│   ├── README.md
│   ├── annotated_bibliography
│   │   ├── real-time.md
│   │   └── ...
│   └── notebooks
│       ├── real-time
│       │   └── rtcloud_notebook.ipynb
│       └── ...
└── rt-cloud
    ├── LICENSE
    ├── Readme.md
    ├── certs
    │   └── ...
    ├── docker
    │   ├── Dockerfile.rtbase
    │   └── Dockerfile.rtcloud
    ├── docs
    │   ├── Firefox-certificate-add-exception.png
    │   └── ...
    ├── environment.yml
    ├── projects
    │   ├── amygActivation
    │   │   └── ...
    │   └── sample
    │       └── ...
    ├── rtCommon
    │   └── validationUtils.py
    ├── scripts
    │   └── ...
    ├── tests
    │   └── ...
    └── web
        └── ...
```

In [1]:
import os
import sys

path_to_rtcloud = '/Users/gwallace/src/github/brainiak/rt-cloud' # insert your path here
sys.path.append(path_to_rtcloud)

path_to_notebook = os.getcwd()  # check and change notebook path as needed

In [2]:
import os
import sys
from rtCommon.projectInterface import Web
from rtCommon.structDict import StructDict

scriptToRun = 'projects/sample/sample.py'
initScript = 'projects/sample/initialize.py'
finalizeScript = 'projects/sample/finalize.py'

# scriptToRun = os.path.join(path_to_notebook, 'sample.py')
# initScript = os.path.join(path_to_notebook, 'initialize.py')
# finalizeScript = os.path.join(path_to_notebook, 'finalize.py')

config = StructDict({
    'title' : 'Sample Project 2',
    'sessionId' : '20180310T123000',
    'subjectName' : '0219191_faceMatching',
    'datestr' : '20190219',
    'subjectNum' : 101,
    'subjectDay' : 1,
    'sessionNum' : 1,
    'runNum' : [1],
    'scanNum' : [13],

    ## Plotting settings
    'plotTitle' : 'Realtime Plot',
    'plotXLabel' : 'Sample #',
    'plotYLabel' : 'Value',
    'plotXRangeLow' : 0,
    'plotXRangeHigh' : 20,
    'plotYRangeLow' : -1,
    'plotYRangeHigh' : 1,
    'plotAutoRangeX' : False,
    'plotAutoRangeY' : True,

    ## important variables about the dicom files
    'dicomNamePattern' : '001_0000{}_000{}.dcm',
    'minExpectedDicomSize' : 300000,
})

params = StructDict({
    'fmriPyScript': scriptToRun,
    'initScript': initScript,
    'finalizeScript': finalizeScript,
    'filesremote': False,
    'port': 8889
})

web = Web()
# web.start(params, config, testMode=True)

/Users/gwallace/src/github/brainiak/rt-cloud/rtCommon/imageHandling.py:19: UserWarning: The DICOM readers are highly experimental, unstable, and only work for Siemens time-series at the moment
Please use with caution.  We would be grateful for your help in improving them
  from nibabel.nicom import dicomreaders


In [6]:
%%script python --bg
web.start(params, config, testMode=True)

* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0

* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0



In [7]:
%%html
<iframe src="http://localhost:8889" width="800" height="600"></iframe>